In [2]:
import numpy as np
import torch
from torch import nn
import math
from sklearn.model_selection import train_test_split

In [5]:
device = torch.device('cuda:1')

In [ ]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, data_inputs, data_targets):
        self.inputs = torch.FloatTensor(data_inputs)
        self.label = torch.FloatTensor(data_targets)

    def __getitem__(self, index):
        return self.inputs[index], self.label[index]

    def __len__(self):
        return len(self.inputs)

In [ ]:
def get_dataloader(features, labels, batch_Size=256):
    x_train, x_test, y_train, y_test = train_test_split(
        features,
        labels,
        test_size=0.2,
        random_state=1,
        shuffle=True,
    )
    train_dataset = DataSet(np.array(x_train), list(y_train))
    test_dataset = DataSet(np.array(x_test), list(y_test))

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_Size, shuffle=True, drop_last=True
    )
    test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_Size, shuffle=True, drop_last=True
    )
    return train_dataloader, test_dataloader

In [23]:
class ClassificationNet(nn.Module):
    def __init__(self,dimension):
        super(ClassificationNet,self).__init__()
        
        self.fc_1 = nn.Linear(dimension,1024)
        self.fc_2 = nn.Linear(1024,1024)
        self.fc_3 = nn.Linear(1024,256)
        self.fc_4 = nn.Linear(256,32)
        self.fc_5 = nn.Linear(32,6)
    
    def forward(self,x):
        x = self.fc_1(x)
        x = torch.relu(x)
        x = self.fc_2(x)
        x = torch.relu(x)
        x = self.fc_3(x)
        x = torch.relu(x)
        x = self.fc_4(x)
        x = torch.relu(x)
        x = self.fc_5(x)
        x = nn.functional.softmax(x,dim=-1)
        
        return x
        

In [ ]:
def train(
    model,
    train_dataloader,
    test_dataloader,
    model_path='/home/hanyuji/projects/PJ4/best_model',
    epochs=200,
):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    best_loss = math.inf
    model.train()

    for epoch in range(epochs):
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # 保存当前最好的模型
        if epoch % 10 == 0:
            cur_loss = validation(model, test_dataloader, criterion)
            if cur_loss < best_loss:
                best_loss = cur_loss

                # 保存模型参数
                weights_dict = {}
                for k, v in model.state_dict().items():
                    weights_dict[k] = v
                torch.save(weights_dict, f'{model_path}/model_best.pkl')

            print(f'epoch: {epoch}, train_loss: {float(loss)}, val_loss: {best_loss}')


def validation(model, test_dataloader, criterion):
    total_lost = []
    model.eval()
    with torch.no_grad():
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            total_lost.append(loss.cpu().numpy())

    return np.array(total_lost).mean()

In [ ]:
train_features = []


In [78]:
arr1 = np.load('/home/hanyuji/projects/PJ4/data/local_features/local_feature_orb_train.npy')
arr2 = np.load('/home/hanyuji/projects/PJ4/data/labels/local_label_orb_train.npy')

In [79]:
(arr1 == arr2)

/home/hanyuji/miniconda3/envs/CV/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [80]:
arr2.shape

(4499,)

In [81]:
arr1.shape

(4499, 50)

In [86]:
np.bincount(arr2)

array([  0, 750, 749, 750, 750, 750, 750])